In [ ]:
import json
import pandas as pd

df = pd.read_csv("yelp_ml_master.csv")

In [ ]:
df.columns

In [ ]:
df = df.dropna()

In [ ]:
import pandas as pd
import ast

# Assuming 'df' is your DataFrame and it has a 'stars_y' column for ratings

# Define a function to safely convert stringified lists to actual lists
def string_to_list(string_list):
    try:
        return ast.literal_eval(string_list)
    except ValueError:
        # In case of an error, return an empty list
        return []
    except SyntaxError:
        # Handle strings that are not in list format (e.g., single quotes missing)
        return [string_list.strip("[]").replace("'", "").split(", ")]

# Apply the function to the entire 'categories_list' column
df['categories_list'] = df['categories_list'].apply(string_to_list)


In [ ]:
df['hours'] = df['hours'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('{') and x.endswith('}') else x)

In [ ]:
def time_to_minutes(time_str):
    # Split the time string into hours and minutes
    hours, minutes = map(int, time_str.split(':'))
    # Convert hours to minutes and add to minutes
    return hours * 60 + minutes


In [ ]:
possible_keys = set()
i = 0
for hours in df['hours']:
    print(hours)
    for day, hour in hours.items():
        str_list = hour.split('-')
        time = time_to_minutes(str_list[1]) - time_to_minutes(str_list[0])
        if time < 0:
            time = time_to_minutes(str_list[1]) + 24 * 60 - time_to_minutes(str_list[0])
            i += 1



In [ ]:
import numpy as np

In [ ]:
def get_total_time(row):
    hours = row['hours']
    total_time = []
    for day, hour in hours.items():
        str_list = hour.split('-')
        time = time_to_minutes(str_list[1]) - time_to_minutes(str_list[0])
        if time < 0:
            time = time_to_minutes(str_list[1]) + 24 * 60 - time_to_minutes(str_list[0])
        if time > 0 : total_time.append(time)
    return np.sum(total_time)

In [ ]:
def get_average_time(row):
    hours = row['hours']
    total_time = []
    for day, hour in hours.items():
        str_list = hour.split('-')
        time = time_to_minutes(str_list[1]) - time_to_minutes(str_list[0])
        if time < 0:
            time = time_to_minutes(str_list[1]) + 24 * 60 - time_to_minutes(str_list[0])
        if time > 0 : total_time.append(time)
    return np.mean(total_time)

In [ ]:
def get_is_overnight(row):
    hours = row['hours']
    total_time = []
    for day, hour in hours.items():
        str_list = hour.split('-')
        time = time_to_minutes(str_list[1]) - time_to_minutes(str_list[0])
        finish_time =  time_to_minutes(str_list[1]) + 24 * 60 if time < 0 else time_to_minutes(str_list[1])
        if finish_time > 22 * 60: return True
    return False

In [ ]:
df['over_midnight'] = df.apply(get_is_overnight, axis=1)

In [ ]:
df['total_time'] = df.apply(get_total_time, axis=1)
df['mean_time'] = df.apply(get_average_time, axis=1)

In [ ]:
df = df.dropna()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df['mean_time'], df['stars_y'], s= 1, alpha= 0.2)

In [ ]:
df_open['total_time'].corr(df['stars_y'])

In [ ]:
plt.scatter(df_open['total_time'], df_open['stars_y'], s= 1, alpha= 0.2)

In [ ]:
df_open = df[df['is_open'] == True]

In [ ]:
df_open.columns

In [ ]:
df_low = df[df['stars_x'] == 1]
df_high = df[df['stars_x'] == 5]

In [ ]:
from scipy.stats import ttest_ind
t_stat, p_value = ttest_ind(df_low['mean_time'], df_high['mean_time'])
print(f"T-statistic: {t_stat}, P-value: {p_value}")

In [ ]:
plt.hist(df_low['mean_time'])

In [ ]:
plt.hist(df_high['mean_time'])

In [ ]:
# Data
mean_time_high = df_high['mean_time']
mean_time_low = df_low['mean_time']

# Histogram settings
bins = np.histogram_bin_edges(mean_time_high, bins='auto')
plt.hist(mean_time_high, bins=bins, alpha=0.5, label='High')
plt.hist(mean_time_low, bins=bins, alpha=0.5, label='Low')

# Adding labels and title
plt.xlabel('Mean Time')
plt.ylabel('Frequency')
plt.title('Side-by-Side Histogram of Mean Time')
plt.legend()

# Display the plot
plt.show()


In [ ]:
# Data
mean_time_high = df_high['mean_time']
mean_time_low = df_low['mean_time']

# Histogram settings with normalization
bins = np.histogram_bin_edges(mean_time_high, bins='auto')
plt.hist(mean_time_high, bins=bins, alpha=0.5, label='Avg.rate = 5', density=True)
plt.hist(mean_time_low, bins=bins, alpha=0.5, label='Avg.rate = 1', density=True)

# Adding labels and title
plt.xlabel('Mean Time')
plt.ylabel('Probability Density')
plt.title('Normalized Side-by-Side Histogram of Mean Time')
plt.legend()

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

plt.figure(figsize=(10, 4))
# Data
mean_time_high = df_high['mean_time']
mean_time_low = df_low['mean_time']

# Histogram settings with normalization
bins = np.histogram_bin_edges(mean_time_high, bins='auto')
plt.hist(mean_time_high, bins=bins, alpha=0.5, label='Avg.rate = 5', density=True)
plt.hist(mean_time_low, bins=bins, alpha=0.5, label='Avg.rate = 1', density=True)

# Fit a Gaussian to the data
mu_high, std_high = norm.fit(mean_time_high)
mu_low, std_low = norm.fit(mean_time_low)

# Plot the Gaussian fit
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p_high = norm.pdf(x, mu_high, std_high)
p_low = norm.pdf(x, mu_low, std_low)

plt.plot(x, p_high, '#0355E3', linewidth=2)
plt.plot(x, p_low, '#FC5400', linewidth=2)

# Adding labels and title
plt.xlabel('Opening Hours / day')
plt.ylabel('Probability Density')
plt.title('Ditribution of Opending hours')

# Display the plot
plt.show()


In [ ]:
df_day = df[df['over_midnight'] == False]
df_night = df[df['over_midnight'] == True]

df_day = df_day[df_day['is_open'] == True]
df_night = df_night[df_night['is_open'] == True]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm


# Data
mean_time_high = df_day['stars_y']
mean_time_low = df_night['stars_y']

# Histogram settings with normalization
bins = np.histogram_bin_edges(mean_time_high, bins='auto')
plt.hist(mean_time_high, bins=bins, alpha=0.5, label= f'Only open in day', density=True)
plt.hist(mean_time_low, bins=bins, alpha=0.5, label= f'Open in mid-night', density=True)

# Fit a Gaussian to the data
mu_high, std_high = norm.fit(mean_time_high)
mu_low, std_low = norm.fit(mean_time_low)

# Plot the Gaussian fit
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p_high = norm.pdf(x, mu_high, std_high)
p_low = norm.pdf(x, mu_low, std_low)

plt.plot(x, p_high, '#0355E3', linewidth=2)
plt.plot(x, p_low, '#FC5400', linewidth=2)

# Adding labels and title
plt.xlabel('Average stars')
plt.ylabel('Probability Density')
plt.legend()

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Example using seaborn's regplot
sns.regplot( x=df['mean_time'], y=df['stars_y'], ci=99, scatter_kws={'s': 1, 'alpha': 0.5})

# Adding labels, title, and legend
plt.xlabel('Index')
plt.ylabel('Mean Time')
plt.title('Regression with Confidence Interval')
plt.legend()

# Display the plot
plt.show()


In [ ]:
sns.regplot

In [ ]:
# Fitting a polynomial of degree 3
polynomial_coefficients = np.polyfit(df['mean_time'], df['stars_y'], 3)
polynomial = np.poly1d(polynomial_coefficients)

# Generating x values and corresponding y values from the polynomial
x_values = np.linspace(df['mean_time'].min(), df['mean_time'].max(), 100)
y_values = polynomial(x_values)


In [ ]:
df.columns

In [ ]:
df['opening_hour'] = df['mean_time'] / 60

In [ ]:
# Plotting the data and the polynomial fit
sns.lmplot(x='distance', y='stars_y', data=df, order=2, ci=99.5, scatter_kws={'s': 0.1, 'alpha': 0.5, 'color': '#6B92D5'}, line_kws={'color': 'red', 'linewidth': 1})

# Alternatively, using regplot with the polynomial line
#sns.regplot(x=df['mean_time'], y=df['stars_y'], scatter_kws={'s': 0.5, 'alpha': 0.1}, fit_reg=False)

# Adding labels and title
plt.xlabel('Distance to Downtown (km)')
plt.ylabel('Average Rating')

plt.xlim(0,24.5)

# Display the plot
plt.show()


In [ ]:
df.sort_values('mean_time')

In [ ]:
brand_counts = df.groupby(['name']).size().reset_index(name='count')

In [ ]:
brand_counts.sort_values('count')

In [ ]:
df = df.merge(brand_counts, on = 'name')

In [ ]:
df['count'].corr(df['stars_y'])

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame

# Group by 'name' and calculate the average of 'stars_y'
grouped_df = df.groupby('name')[['stars_y','count']].mean()

# The result is a Series with the 'name' as the index and the average 'stars_y' as the value
# If you want to convert it back to a DataFrame:
grouped_df = grouped_df.reset_index()

# Now 'grouped_df' is a DataFrame with two columns: 'name' and the average of 'stars_y'
grouped_df = grouped_df[grouped_df['count'] > 100]
plt.scatter(grouped_df['count'], grouped_df['stars_y'],  marker='^', alpha= 1, color = "red")
plt.ylim(0,5)

In [ ]:
df.columns

In [ ]:
df['invest']

In [ ]:
# Plotting the data and the polynomial fit
sns.lmplot(x='count', y='review_count', data=df, order=2, ci=99.5, scatter_kws={'s': 1, 'alpha': 0.1, 'color': '#6B92D5'}, line_kws={'color': 'red', 'linewidth': 1})

# Alternatively, using regplot with the polynomial line
#sns.regplot(x=df['mean_time'], y=df['stars_y'], scatter_kws={'s': 0.5, 'alpha': 0.1}, fit_reg=False)
#plt.scatter(grouped_df['count'], grouped_df['stars_y'],  marker='^', alpha= 1, color = "red")

# Adding labels and title
plt.xlabel('Number of resturant in the brand')
plt.ylabel('Average star')

# Display the plot
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
# Plotting the data and the polynomial fit
sns.regplot(x='Count', y='AverageRating', data=category_stats_df, order=1, ci=95,scatter_kws={'s': 8, 'alpha': 1, 'color': '#3761A9'}, line_kws={'color': 'red', 'linewidth': 1})
#plt.scatter(category_stats_df['Count'], category_stats_df['AverageRating'], color='#3761A9', s = 5)
# Adding labels and title
plt.xlabel('Number of resturant in the category')
plt.ylabel('Average star') 

#plt.ylim(0.5, 5.5)
# Display the plot
plt.show()

In [ ]:
df

In [ ]:
brand_counts.sort_values('count')

In [ ]:
brand_counts_filter = brand_counts[brand_counts['count'] > 150]

In [ ]:
brand_counts_filter

In [ ]:
brand_counts_average = df.groupby(['name']).size().reset_index(name='count')

In [ ]:
df['categories_list'] 

In [ ]:
category_average_df = category_average.reset_index()

In [ ]:

# Now, you can explode the 'categories_list' column
exploded_df = df.explode('categories_list')

# Group by the individual categories and calculate the mean rating and count
category_stats = exploded_df.groupby('categories_list')['stars_y'].agg(['mean', 'count'])

# Convert the Series with multi-level columns to a DataFrame if needed
category_stats_df = category_stats.reset_index()

# Rename columns for clarity
category_stats_df.columns = ['Category', 'AverageRating', 'Count']

# Now 'category_stats_df' is a DataFrame with 'Category', 'AverageRating', and 'Count' as columns


In [ ]:

# Now, you can explode the 'categories_list' column
exploded_df = df.explode('categories_list')

# Group by the individual categories and calculate the mean rating and count
category_stats = exploded_df.groupby('categories_list')['review_count'].agg(['mean', 'count'])

# Convert the Series with multi-level columns to a DataFrame if needed
category_stats_df = category_stats.reset_index()

# Rename columns for clarity
category_stats_df.columns = ['Category', 'ReviewCountAverage', 'Count']

# Now 'category_stats_df' is a DataFrame with 'Category', 'AverageRating', and 'Count' as columns

In [ ]:
category_stats_df

In [ ]:
plt.scatter(category_stats_df['Count'],category_stats_df['ReviewCountAverage'] )

In [ ]:
category_stats_df.sort_values('Count')

In [ ]:
category_stats_df = category_stats_df

In [ ]:
category_stats_df = category_stats_df[category_stats_df['Count'] >  1500]
category_stats_df = category_stats_df[category_stats_df['Count'] <  7370]


In [ ]:
category_stats_df 

In [ ]:
category_stats_df = category_stats_df[category_stats_df['Count'] > 7370]

In [ ]:
category_stats_df['Count'].corr(category_stats_df['AverageRating'])


In [ ]:
category_stats_df['Count']

In [ ]:
exploded_df['categories_list']

In [ ]:
df_city = pd.read_csv("worldcities.csv")

In [ ]:
df_city 

In [ ]:
df_city = df_city[df_city['country'] == "United States"]

In [ ]:
df_city 

In [ ]:
df = df.merge(df_city, on = "city")

In [ ]:
df = df.rename(columns={'lat_y': 'lat_city', 'lng':"lng_city"})

In [ ]:
df.columns

In [ ]:
df['distance'] = np.sqrt((df['latitude'] - df['lat_city'] ) ** 2 + (df['longitude'] - df['lng_city'])**2) * 111

In [ ]:
plt.scatter(df['distance'], df['stars_y'])

In [ ]:
df.sort_values("distance")

In [ ]:
df_result['distance'].corr(df_result['stars_y'])

In [ ]:
df.columns

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# First, sort the DataFrame based on 'business_id' and 'distance' in descending order
df_sorted = df.sort_values(by=['business_id', 'distance'], ascending=[True, True])

# Then, group by 'business_id' and take the first entry of each group
df_result = df_sorted.groupby('business_id').first().reset_index()

# df_result now contains only one row for each business_id with the largest distance


In [ ]:
df.columns

In [ ]:
df.to_csv("relationship_data.csv")

In [ ]:
from scipy.stats import linregress
cols = ['distance', 'me\', 'population_city', 'count']

for col in cols:
    slope, intercept, r_value, p_value, std_err = linregress(df[col], df['stars_y'])
    print(f"{col}, {slope} {intercept} {r_value} {p_value} {std_err}")
    plt.figure(figsize=(6, 6))
    # Plotting the data and the polynomial fit
    sns.regplot(x='Count', y='AverageRating', data=category_stats_df, order=1, ci=95,scatter_kws={'s': 8, 'alpha': 1, 'color': '#3761A9'}, line_kws={'color': 'red', 'linewidth': 1})
    #plt.scatter(category_stats_df['Count'], category_stats_df['AverageRating'], color='#3761A9', s = 5)
    # Adding labels and title
    plt.xlabel({col})
    plt.ylabel('Average star') 
    plt.show()


In [ ]:
for col in cols:
    slope, intercept, r_value, p_value, std_err = linregress(df[col], df['stars_y'])
    print(f"{col}, {slope} {intercept} {r_value} {p_value} {std_err}")

In [ ]:
df.columns

In [ ]:
df_filter = df[df['review_count'] > 100]

for col in cols:
    slope, intercept, r_value, p_value, std_err = linregress(df_filter[col], df_filter['stars_y'])
    print(f"{col}, {slope} {intercept} {r_value} {p_value} {std_err}")

In [ ]:
category_stats_df['Count'].corr(category_stats_df['Count'])

slope, intercept, r_value, p_value, std_err = linregress(category_stats_df['opening_hour'], category_stats_df['ReviewCountAverage'])
print(f"{col}, {slope} {intercept} {r_value} {p_value} {std_err}")

In [ ]:
category_stats_df


In [ ]:
plt.scatter(df['density'], df['stars_y'], s= 0.1)

In [ ]:
df['density']

In [ ]:
df.columns